In [ ]:
"""
PM2.5 Forecasting Data Pipeline - Bangkok
Run this notebook to collect and process data for ST-UNN model training.

✅ AI Engineering Standards:
- Custom exception hierarchy
- Data validation at boundaries
- Comprehensive error handling
- Structured logging
- Type hints throughout
"""
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from pipeline import PM25Pipeline
from config import PipelineConfig
from datetime import datetime, timedelta
import pandas as pd
from core.exceptions import PipelineError

print("🚀 PM2.5 Forecasting Data Pipeline")
print("=" * 60)
print("✅ AI Engineering & Data Science Standards Applied")
print("=" * 60)


In [ ]:
# Initialize pipeline
config = PipelineConfig()
pipeline = PM25Pipeline(config)

print("✅ Pipeline initialized")
print(f"📁 Data directory: {config.storage.BASE_DIR}")
print(f"🌍 Bangkok bbox: {config.data.BANGKOK_BBOX}")


In [ ]:
# Set date range (ตั้งแต่ปี 2010)
end_date = datetime.utcnow()
start_date = datetime(2010, 1, 1)  # เริ่มตั้งแต่ปี 2010

print(f"📅 Date range: {start_date.date()} to {end_date.date()}")
print(f"📊 Total days: {(end_date - start_date).days} days")
print(f"📊 Years: {(end_date - start_date).days / 365:.1f} years")


In [ ]:
# Run complete pipeline
result = pipeline.run(
    start_date=start_date,
    end_date=end_date,
    save_intermediate=True
)

print(f"\n✅ Pipeline completed!")
print(f"📊 Total records: {len(result)}")
print(f"🏢 Stations: {result['station_id'].nunique() if not result.empty else 0}")


In [ ]:
# Inspect results
if not result.empty:
    print("\n📈 Dataset Summary:")
    print(result.info())
    print("\n📊 First few records:")
    print(result.head())
    print("\n📉 Statistics:")
    print(result.describe())


In [ ]:
# Create sliding windows for ST-UNN training
from utils.sliding_window import create_sliding_window

# Define feature columns
feature_cols = [
    "temperature", "humidity", "pressure",
    "u_wind", "v_wind", "precipitation", "solar",
    "fire_count", "land_use", "population_density",
    "hour_sin", "hour_cos", "day_of_week", "month"
]

# Filter available features
available_features = [col for col in feature_cols if col in result.columns]
print(f"✅ Available features: {available_features}")

# Create sequences
X, y = create_sliding_window(
    result,
    input_hours=config.model.INPUT_HOURS,
    output_hours=config.model.OUTPUT_HOURS,
    feature_cols=available_features,
    target_col="pm25"
)

print(f"\n🎯 Sequences created:")
print(f"   X shape: {X.shape}")
print(f"   y shape: {y.shape}")
print(f"   Ready for ST-UNN training! 🚀")
